In [1]:
import os
import re
import sys
import time
import implicit
import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import  tqdm

In [3]:
IS_LOCAL = True
HOME_DIR = ('/mnt/E/Projects/' if IS_LOCAL else '/home/Xetd71/') + 'Content-based-Neural-Recommender-Systems/'
os.environ['HOME_DIR'] = HOME_DIR

sys.path.append("../..")
from utils.prepare_data import zen, tokenize
from utils.evaluate import test
from utils import tqdm_utils

DATA_DIR = f'{HOME_DIR}data/zen/'
# WORKING_DIR = f'{HOME_DIR}models/dssm'
# os.chdir(WORKING_DIR)

100%|██████████| 4349/4349 [00:12<00:00, 340.64it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/E/Projects/Content-based-Neural-Recommender-Systems/models/dssm'

# Load data

In [3]:
users_train_df, users_test_df = zen.split_users_df()

loading users: 42977it [02:19, 307.93it/s]


In [4]:
items_df = zen.items_df()

loading items: 328050it [00:33, 9790.91it/s] 


In [9]:
N_USERS = 42977

In [5]:
PREPROC_DIR = f'{DATA_DIR}preproc/'

In [23]:
def get_user_item_rating_matrix(users_df):
    user_ids = np.array([np.concatenate([[v]*v_len for v, v_len in zip(
        users_df['userId'].values, users_df['userItems'].apply(len).values)])], dtype=np.int).T
    items_ids = np.array([np.concatenate(users_df['userItems'].values)], dtype=np.int).T
    ratings = np.array([np.concatenate(users_df['userRatings'].values)], dtype=np.int).T
    return np.concatenate((user_ids, items_ids, ratings), axis=1)

In [24]:
ratings_matrix = get_user_item_rating_matrix(users_test_df)

In [14]:
# # items
# items_processed_text = np.load(f'{PREPROC_DIR}items_processed_text.npy')
# items_processed_images = np.load(f'{PREPROC_DIR}items_processed_images.npy')
# print(items_processed_text.shape, items_processed_images.shape)

# # ratings
# ratings_matrix = np.load(f'{PREPROC_DIR}ratings_matrix.npy')
ratings_0_matrix = np.load(f'{PREPROC_DIR}ratings_0_matrix.npy')
ratings_1_matrix = np.load(f'{PREPROC_DIR}ratings_1_matrix.npy')
print(ratings_matrix.shape)

(67780168, 3)


In [15]:
w0, w1 = 0.75, 0.25

In [16]:
user_items_ids = {}
for user, item, _ in tqdm(ratings_1_matrix):
    if user in user_items_ids:
        user_items_ids[user].append(item)
    else:
        user_items_ids[user] = [item]

100%|██████████| 6450655/6450655 [00:12<00:00, 529192.38it/s]


In [17]:
items_matrix = np.concatenate([items_processed_text, items_processed_images], axis=1)
items_matrix.shape

(328050, 192)

In [18]:
users_matrix = []
# user_n_clicks = []
for user in tqdm(np.arange(N_USERS)):
    user_items = items_matrix[user_items_ids[user]]
#     user_n_clicks.append(user_items.shape[0])
    users_matrix.append(user_items.mean(axis=0))
users_matrix = np.vstack(users_matrix)
# user_n_clicks = np.array(user_n_clicks)

100%|██████████| 42977/42977 [00:03<00:00, 11698.83it/s]


In [19]:
users_matrix.shape

(42977, 192)

In [20]:
np.save(f'{PREPROC_DIR}users_embeddings.npy', users_matrix)

In [21]:
np.save(f'{PREPROC_DIR}items_embeddings.npy', items_matrix)

In [12]:
VOCAB_SIZE = users_matrix.shape[1]

In [57]:
def ratings_batch(ratings_matrix=ratings_matrix, batch_size=64):
    idxs = np.random.randint(0, ratings_matrix.shape[0], batch_size)
    items = ratings_matrix[idxs]
    
    return users_matrix[items[:, 0]], items_matrix[items[:, 1]], items[:, 2].T

def normalized_ratings_batch(w0=0.5, batch_size=64):
    split_size = int(batch_size*w0)
    return tuple(map(lambda x: np.concatenate((x[0], x[1])), zip(
        ratings_batch(ratings_0_matrix, split_size), 
        ratings_batch(ratings_1_matrix, batch_size - split_size)
    )))

## Train model

In [58]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.callbacks import TensorBoard

In [59]:
# reset graph when you change architecture!
def reset_tf_session():
    curr_session = tf.get_default_session()
    # close current session
    if curr_session is not None:
        curr_session.close()
    # reset graph
    K.clear_session()
    # create new session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    K.set_session(s)
    return s

In [60]:
# import necessary building blocks
from keras.models import Model
from keras.layers import Dense, Input, Embedding, dot, Reshape, Dropout, Activation, Conv1D

In [61]:
def make_model():
    user_embedding = Input(dtype='float32', shape=(VOCAB_SIZE,), name='user_embedding') 
    item_embedding = Input(dtype='float32', shape=(VOCAB_SIZE,), name='item_embedding')
    
    user_layer = Dense(300, activation="elu", input_shape=(VOCAB_SIZE,))(user_embedding)
#     user_layer = Dropout(0.2)(user_layer)
    user_layer = Dense(300, activation="elu", input_shape=(300,))(user_layer)
#     user_layer = Dropout(0.2)(user_layer)
    user_layer = Dense(128, activation=None, input_shape=(300,))(user_layer)
    
    item_layer = Dense(300, activation="elu", input_shape=(VOCAB_SIZE,))(item_embedding)
    item_layer = Dropout(0.2)(item_layer)
    item_layer = Dense(300, activation="elu", input_shape=(300,))(item_layer)
    item_layer = Dropout(0.2)(item_layer)
    item_layer = Dense(128, activation=None, input_shape=(300,))(item_layer)
    
    user_item_product = dot([user_layer, item_layer], axes=1)
    pred_y = Activation("sigmoid")(user_item_product)
    
    model = Model(inputs=[user_embedding, item_embedding], outputs=pred_y)
    
    return model

In [62]:
# describe model
s = reset_tf_session()  # clear default graph
model = make_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
item_embedding (InputLayer)     (None, 192)          0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 300)          57900       item_embedding[0][0]             
__________________________________________________________________________________________________
user_embedding (InputLayer)     (None, 192)          0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 300)          0           dense_4[0][0]                    
__________________________________________________________________________________________________
dense_1 (D

In [93]:
BATCH_SIZE = 512
STEPS_PER_EPOCH = 40_000
EPOCHS = 100

s = reset_tf_session()  # clear default graph
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.adam(lr=0.0001),  # gradient clipping just in case
)

In [94]:
# for saving the model after every epoch
from keras.models import save_model

class ModelSaveCallback(keras.callbacks.Callback):
    def __init__(self, file_name, period):
        super(ModelSaveCallback, self).__init__()
        self.file_name = file_name
        self.period = period

    def on_epoch_end(self, epoch, logs):
        model_filename = self.file_name.format(epoch)
        if (epoch + 1) % self.period == 0:
            save_model(self.model, model_filename)
            print(f"Model saved in {model_filename}")

In [95]:
def dssm_predict(user_id, items):
    return model.predict([users_matrix[[user_id]*len(items)],  items_matrix[items]])[:, 0]

def prediction_hist(p, bins=5):
    hist = []
    for th in np.linspace(0, 1-1/bins, bins):
        hist.append(np.logical_and(th <= p, p <= (th+1/bins)).sum())
    return hist

def dssm_get_prediction_hist(bins=5):
    user_embedding, item_embeddings, ratings = normalized_ratings_batch(batch_size=BATCH_SIZE)
    return prediction_hist(model.predict([user_embedding, item_embeddings])[:, 0], bins)

In [96]:
# for saving the model after every epoch
from keras.models import save_model

class ModelLogFileCallback(keras.callbacks.Callback):
    def __init__(self, file_name):
        super(ModelLogFileCallback, self).__init__()
        self.file_name = file_name
        
    def on_train_begin(self, logs):
        self.file = open(self.file_name, 'a+')
        self.epoch_time = time.time()
 
    def on_train_end(self, logs):
        self.file.close()

    def on_epoch_end(self, epoch, logs):
        ndcg = test.ndcg(dssm_predict)
        p_hist = dssm_get_prediction_hist()
        cur_time = time.time()
        logs['ndcg'] = ndcg
        msg = "Epoch {},   loss: {:.4f},   val ndcg: {:.4f},   p_hist: [{}],   time: {:.4f}".format(
            epoch,
            logs.get('loss'),
            ndcg,
            ', '.join(list(map(str, p_hist))),
            cur_time - self.epoch_time,
        )
        print(msg)
        print(msg, file=self.file, flush=True)
        self.epoch_time = cur_time

In [97]:
last_finished_epoch = 0

# you can continue from snapshot!!!
# from keras.models import load_model
# s = reset_tf_session()
# last_finished_epoch = 103
# model = load_model(path("model4/model_{}".format(last_finished_epoch)), 
#                    custom_objects={"top_3_accuracy": top_3_accuracy})

In [98]:
def train_iterator(w0, batch_size):
    while True:
        user_id, item_embeddings, ratings = normalized_ratings_batch(w0, batch_size=batch_size)
#         user_id, item_embeddings, ratings = ratings_batch(batch_size=batch_size, ratings_matrix=ratings_matrix)
        yield [user_id, item_embeddings], ratings 

In [99]:
import time
tensorboard = TensorBoard(log_dir=f"/data/home/Xetd71/DSSM2/d2v+img/logs/{time.time()}")

In [100]:
1/w0

1.3333333333333333

In [101]:
1/w1

4.0

In [102]:
# fit the model with our eternal generator!
model.fit_generator(
    train_iterator(w0, BATCH_SIZE), 
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    callbacks=[
        ModelSaveCallback("/data/home/Xetd71/DSSM2/d2v+img/model_{}", 1),
        ModelLogFileCallback("/data/home/Xetd71/DSSM2/d2v+img/logs.txt"),
        tensorboard,
    ],
    verbose=1,
    initial_epoch=last_finished_epoch,
    class_weight={0: 1/w0/2, 1: 1/w1/2}
)

Epoch 1/100
40000/40000 [==============================] - 288s 7ms/step - loss: 0.6476
Model saved in /data/home/Xetd71/DSSM2/d2v+img/model_0
Epoch 0,   loss: 0.6476,   val ndcg: 0.2286,   p_hist: [14, 132, 228, 120, 18],   time: 402.9810
Epoch 2/100
40000/40000 [==============================] - 290s 7ms/step - loss: 0.6285
Model saved in /data/home/Xetd71/DSSM2/d2v+img/model_1
Epoch 1,   loss: 0.6285,   val ndcg: 0.2288,   p_hist: [20, 110, 233, 131, 18],   time: 405.3160
Epoch 3/100
40000/40000 [==============================] - 289s 7ms/step - loss: 0.6180
Model saved in /data/home/Xetd71/DSSM2/d2v+img/model_2
Epoch 2,   loss: 0.6180,   val ndcg: 0.2313,   p_hist: [34, 122, 192, 144, 20],   time: 403.7675
Epoch 4/100
40000/40000 [==============================] - 287s 7ms/step - loss: 0.6107
Model saved in /data/home/Xetd71/DSSM2/d2v+img/model_3
Epoch 3,   loss: 0.6107,   val ndcg: 0.2288,   p_hist: [36, 104, 193, 154, 25],   time: 401.6559
Epoch 5/100
40000/40000 [===============

KeyboardInterrupt: 

# test2

In [70]:
items_matrix = items_processed_text
items_matrix.shape

(328050, 96)

In [71]:
users_matrix = []
for user in tqdm(np.arange(N_USERS)):
    users_matrix.append(items_matrix[user_items_ids[user]].mean(axis=0))
users_matrix = np.vstack(users_matrix)

100%|██████████| 42977/42977 [00:02<00:00, 20295.53it/s]


In [76]:
VOCAB_SIZE = users_matrix.shape[1]
BATCH_SIZE = 512
STEPS_PER_EPOCH = 40_000
EPOCHS = 20

s = reset_tf_session()  # clear default graph
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.adam(),  # gradient clipping just in case
)

In [77]:
import time
tensorboard = TensorBoard(log_dir=f"/data/home/Xetd71/DSSM2/d2v/logs/{time.time()}")

In [78]:
# fit the model with our eternal generator!
model.fit_generator(
    train_iterator(BATCH_SIZE), 
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    callbacks=[
        ModelSaveCallback("/data/home/Xetd71/DSSM2/d2v/model_{}", 1),
        ModelLogFileCallback("/data/home/Xetd71/DSSM2/d2v/logs.txt"),
        tensorboard,
    ],
    verbose=1,
    initial_epoch=last_finished_epoch,
#     class_weight={0: w0, 1: w1}
)

Epoch 1/20
 6271/40000 [===>..........................] - ETA: 3:38 - loss: 0.6760

KeyboardInterrupt: 

# test3

In [ ]:
items_matrix = items_processed_images
items_matrix.shape

In [ ]:
users_matrix = []
for user in tqdm(np.arange(N_USERS)):
    users_matrix.append(items_matrix[user_items_ids[user]].mean(axis=0))
users_matrix = np.vstack(users_matrix)

In [ ]:
import time
tensorboard = TensorBoard(log_dir=f"/data/home/Xetd71/DSSM2/img/logs/{time.time()}")

In [ ]:
# fit the model with our eternal generator!
model.fit_generator(
    train_iterator(BATCH_SIZE), 
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    callbacks=[
        ModelSaveCallback("/data/home/Xetd71/DSSM2/img/model_{}", 1),
        ModelLogFileCallback("/data/home/Xetd71/DSSM2/img/logs.txt"),
        tensorboard,
    ],
    verbose=1,
    initial_epoch=last_finished_epoch,
#     class_weight={0: w0, 1: w1}
)